<a href="https://colab.research.google.com/github/Pathway2008/CarPrice/blob/main/model_choice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
1. kfold = 5 로 다시 예측모델 비교 = cat+lgb / IONIQ = cat

2. 배터리용량 nan 값 채우기

3. IONIQ 차량 가격 예측을 어떻게 할 것인가?
'''

'\n1. kfold = 5 로 다시 예측모델 비교\n\n2. 배터리용량 nan 값 채우기\n\n3. IONIQ 차량 가격 예측을 어떻게 할 것인가?\n'

In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.7 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

from sklearn.base import RegressorMixin, BaseEstimator
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import shap

In [5]:
train = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/train.csv')
test = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/sample_submission.csv')

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   object 
 2   모델        7497 non-null   object 
 3   차량상태      7497 non-null   object 
 4   배터리용량     4786 non-null   float64
 5   구동방식      7497 non-null   object 
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   object 
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 644.4+ KB


In [7]:
ordinal_features = ['제조사', '차량상태', '구동방식','사고이력',  '모델']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [8]:
model_mapping = dict(enumerate(le.classes_))
model_mapping

{0: 'EV6',
 1: 'ID4',
 2: 'ION5',
 3: 'ION6',
 4: 'IONIQ',
 5: 'KNE',
 6: 'M3',
 7: 'MS',
 8: 'MX',
 9: 'MY',
 10: 'Niro',
 11: 'Q4eT',
 12: 'RSeTGT',
 13: 'Soul',
 14: 'Tay',
 15: 'TayCT',
 16: 'TayGTS',
 17: 'eT',
 18: 'i3',
 19: 'i5',
 20: 'iX'}

In [29]:
# All_data
X_train_all = train.drop(columns=['ID', '가격(백만원)'])
y_train_all = train['가격(백만원)']
model_column_1 = train['모델']

In [47]:
class SklearnCompatibleRegressor(RegressorMixin, BaseEstimator):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# 모델 정의
xgb_model = SklearnCompatibleRegressor(XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=41))
cat_model = SklearnCompatibleRegressor(CatBoostRegressor(n_estimators=500, learning_rate=0.05, random_state=41, verbose=0))
lgb_model = SklearnCompatibleRegressor(LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=41))

# Voting Regressors 정의
voting_models = {
    'XGB+Cat': VotingRegressor([('xgb', xgb_model), ('cat', cat_model)]),
    'XGB+LGB': VotingRegressor([('xgb', xgb_model), ('lgb', lgb_model)]),
    'Cat+LGB': VotingRegressor([('cat', cat_model), ('lgb', lgb_model)]),
    'XGB+Cat+LGB': VotingRegressor([('xgb', xgb_model), ('cat', cat_model), ('lgb', lgb_model)])
}

In [48]:
n_splits = 5
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=41)

rmse_results = {
    'XGB': [],
    'CatBoost': [],
    'LGB': [],
    'XGB+Cat': [],
    'XGB+LGB': [],
    'Cat+LGB': [],
    'XGB+Cat+LGB': []
}

# 결과 저장용 리스트
fold_stats_list = []
fold_stats_notnan_list = []
fold_stats_isnan_list = []

# Stratified K-Fold 실행
for fold, (train_idx, valid_idx) in enumerate(stratified_kfold.split(X_train_all, model_column_1)):
    print(f"Fold {fold + 1}")

    # 훈련 및 검증 데이터 분리
    X_train_fold = X_train_all.iloc[train_idx]
    X_valid_fold = X_train_all.iloc[valid_idx]
    y_train_fold = y_train_all.iloc[train_idx]
    y_valid_fold = y_train_all.iloc[valid_idx]

    # 개별 모델 학습
    xgb_model.fit(X_train_fold, y_train_fold)
    lgb_model.fit(X_train_fold, y_train_fold)
    cat_model.fit(X_train_fold, y_train_fold)

    # Voting 모델 학습
    for name, voting_model in voting_models.items():
        voting_model.fit(X_train_fold, y_train_fold)

    # 예측 수행
    y_pred_xgb = xgb_model.predict(X_valid_fold)
    y_pred_lgb = lgb_model.predict(X_valid_fold)
    y_pred_cat = cat_model.predict(X_valid_fold)

    # 결과 저장 DataFrame 생성
    result_df = pd.DataFrame(X_valid_fold)
    result_df['가격(백만원)'] = y_valid_fold.values
    result_df['XGB_Pred'] = y_pred_xgb
    result_df['CatBoost_Pred'] = y_pred_cat
    result_df['LGB_Pred'] = y_pred_lgb

    # 각 Voting 모델에 대한 예측
    for name, voting_model in voting_models.items():
        result_df[f'{name}_Pred'] = voting_model.predict(X_valid_fold)

    # RMSE 계산
    result_df['xgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB_Pred'])**2)
    result_df['cat-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)
    result_df['lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['LGB_Pred'])**2)
    result_df['xgb_cat-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB+Cat_Pred'])**2)
    result_df['xgb_lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB+LGB_Pred'])**2)
    result_df['cat_lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['Cat+LGB_Pred'])**2)
    result_df['xgb_cat_lgb-real'] = np.sqrt((result_df['가격(백만원)'] - result_df['XGB+Cat+LGB_Pred'])**2)

    # 모델별 RMSE 계산
    rmse_results['XGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB_Pred'])))
    rmse_results['CatBoost'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['CatBoost_Pred'])))
    rmse_results['LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['LGB_Pred'])))
    rmse_results['XGB+Cat'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB+Cat_Pred'])))
    rmse_results['XGB+LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB+LGB_Pred'])))
    rmse_results['Cat+LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['Cat+LGB_Pred'])))
    rmse_results['XGB+Cat+LGB'].append(np.sqrt(mean_squared_error(result_df['가격(백만원)'], result_df['XGB+Cat+LGB_Pred'])))

    # 모델별 통계 계산
    fold_stats = result_df.groupby('모델')[['xgb-real', 'cat-real', 'lgb-real',
                                            'xgb_cat-real', 'xgb_lgb-real',
                                            'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])
    fold_stats_list.append(fold_stats)

    # 배터리 용량 기반 분리
    result_notnan = result_df[~result_df['배터리용량'].isna()]
    result_isnan = result_df[result_df['배터리용량'].isna()]

    fold_stats_notnan = result_notnan.groupby('모델')[['xgb-real', 'cat-real', 'lgb-real',
                                                       'xgb_cat-real', 'xgb_lgb-real',
                                                       'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])
    fold_stats_isnan = result_isnan.groupby('모델')[['xgb-real', 'cat-real', 'lgb-real',
                                                     'xgb_cat-real', 'xgb_lgb-real',
                                                     'cat_lgb-real', 'xgb_cat_lgb-real']].agg(['mean', 'std'])

    fold_stats_notnan_list.append(fold_stats_notnan)
    fold_stats_isnan_list.append(fold_stats_isnan)

Fold 1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000613 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 402
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.327707
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000547 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 402
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 9
[LightGBM] [Info] Start training from score 62.327707
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000899 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total

In [46]:
score_40 = pd.DataFrame(rmse_results)
score_40.loc['Mean'] = score_40.mean()
score_40

,XGB,CatBoost,LGB,XGB+Cat,XGB+LGB,Cat+LGB,XGB+Cat+LGB
0,1.377828,1.394131,1.341322,1.347616,1.343421,1.339355,1.334072
1,1.362897,1.353376,1.449187,1.327709,1.373698,1.352539,1.338640
2,1.481492,1.381758,1.410714,1.393557,1.432591,1.366965,1.388743
3,1.364709,1.388296,1.346675,1.340426,1.338596,1.338548,1.329906
4,1.411122,1.460830,1.381192,1.404182,1.382286,1.394270,1.385350
Mean,1.399610,1.395678,1.385818,1.362698,1.374119,1.358335,1.355342


In [49]:
score_41 = pd.DataFrame(rmse_results)
score_41.loc['Mean'] = score_41.mean()
score_41

,XGB,CatBoost,LGB,XGB+Cat,XGB+LGB,Cat+LGB,XGB+Cat+LGB
0,1.444640,1.399525,1.395832,1.394717,1.409353,1.373609,1.385611
1,1.404549,1.371667,1.342196,1.351491,1.356673,1.331776,1.337791
2,1.411801,1.358950,1.374401,1.357985,1.376773,1.341933,1.351244
3,1.325819,1.326124,1.328226,1.295251,1.313079,1.307663,1.298151
4,1.404262,1.394099,1.400931,1.359740,1.381421,1.368958,1.360024
Mean,1.398214,1.370073,1.368317,1.351837,1.367460,1.344788,1.346564


In [43]:
score_42 = pd.DataFrame(rmse_results)
score_42.loc['Mean'] = score_42.mean()
score_42

,XGB,CatBoost,LGB,XGB+Cat,XGB+LGB,Cat+LGB,XGB+Cat+LGB
0,1.408711,1.369066,1.416139,1.358613,1.398309,1.365161,1.366048
1,1.580166,1.493030,1.485889,1.505723,1.519684,1.464616,1.488961
2,1.359243,1.326666,1.328724,1.308436,1.330777,1.298934,1.304169
3,1.385310,1.372079,1.372442,1.345306,1.365763,1.342838,1.342808
4,1.379405,1.375688,1.347659,1.339493,1.343124,1.334688,1.329453
Mean,1.422567,1.387306,1.390171,1.371514,1.391531,1.361247,1.366288


In [40]:
score = pd.DataFrame(rmse_results)
score.loc['Mean'] = score.mean()
score

,XGB,CatBoost,LGB,XGB+Cat,XGB+LGB,Cat+LGB,XGB+Cat+LGB
0,1.293769,1.349477,1.326646,1.284144,1.295976,1.304414,1.285189
1,1.463144,1.358333,1.431788,1.372944,1.426655,1.355759,1.374277
2,1.410424,1.433234,1.374066,1.386805,1.378672,1.376128,1.371911
3,1.405380,1.360733,1.348042,1.347732,1.359010,1.329542,1.336662
4,1.410476,1.447782,1.433580,1.400824,1.407259,1.415360,1.400121
Mean,1.396638,1.389912,1.382824,1.358490,1.373514,1.356241,1.353632


In [51]:
# 전체 평균 계산
model_stats_mean = pd.concat(fold_stats_list).groupby('모델').mean()
model_stats_notnan_mean = pd.concat(fold_stats_notnan_list).groupby('모델').mean()
model_stats_isnan_mean = pd.concat(fold_stats_isnan_list).groupby('모델').mean()

In [21]:
from IPython.display import display

# 결과 출력
print("Fold별 모델 성능 통계:")
for i, stats in enumerate(fold_stats_list, 1):
    print(f"\nFold {i} stats:")
    display(stats)

Fold별 모델 성능 통계:

Fold 1 stats:


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.526327  0.359750  0.514521  0.344560  0.515969  0.354634     0.515773   
1   0.566369  0.340596  0.598063  0.332313  0.598372  0.345951     0.573667   
2   0.269059  0.186132  0.263631  0.228442  0.299324  0.210361     0.254305   
3   0.275070  0.189922  0.343636  0.167567  0.289185  0.190877     0.304621   
4   5.087022  2.645590  5.047922  2.682258  5.210141  3.079470     5.067472   
5   0.522141  0.308148  0.535969  0.359754  0.537217  0.332848     0.520867   
6   0.368489  0.291218  0.382074  0.307376  0.384896  0.308085     0.356831   
7   0.565883  0.394471  0.624588  0.379185  0.587309  0.375063     0.571898   
8   0.545485  0.348161  0.505074  0.352637  0.620019  0.685152     0.495752   
9   0.541496  0.335213  0.651618  0.435130  0.611899  0.440516     0.558887   
10  0.604630  0.301132  0.587132  0.340004  0.607866  0.320810     0.593229   
11  0.318254  0.222545  0.348328  0.276018  0.323858  0.207891     0.312524   
12  0.288429  0.206174  0.401116  0.397449  0.301315  0.226777     0.312323   
13  0.527455  0.332885  0.587215  0.361764  0.529601  0.329395     0.539724   
14  2.431409  2.088915  2.513941  1.688283  2.324906  1.888125     2.438868   
15  2.625730  2.665106  2.641156  2.099989  2.650919  2.457799     2.601443   
16  0.430601  0.337414  0.631410  0.549022  0.468410  0.380172     0.476170   
17  0.342268  0.270267  0.397407  0.286047  0.388234  0.694445     0.353954   
18  0.291575  0.215896  0.364514  0.303674  0.295452  0.208662     0.298232   
19  0.510561  0.343173  0.499822  0.380780  0.515594  0.296351     0.482711   
20  0.437103  0.314629  0.433728  0.314495  0.493655  0.280198     0.425236   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.348545     0.518799  0.355028     0.510937  0.348958         0.514188   
1   0.320216     0.579554  0.333726     0.590379  0.325838         0.580316   
2   0.192704     0.278488  0.185999     0.266734  0.204445         0.263897   
3   0.163348     0.278779  0.180794     0.311328  0.161840         0.297661   
4   2.446924     5.148582  2.751285     5.129031  2.762531         5.115028   
5   0.323650     0.527997  0.311074     0.527941  0.324838         0.524063   
6   0.287923     0.366353  0.295442     0.349339  0.285575         0.353702   
7   0.358356     0.569191  0.373972     0.577133  0.360231         0.565506   
8   0.352461     0.580429  0.434002     0.540221  0.431301         0.532549   
9   0.363372     0.574327  0.371784     0.580364  0.399624         0.564066   
10  0.290300     0.602725  0.305088     0.590631  0.302300         0.594076   
11  0.245519     0.316221  0.210384     0.314170  0.237647         0.312543   
12  0.258628     0.291449  0.204418     0.320230  0.273917         0.297322   
13  0.321046     0.526427  0.316899     0.539429  0.320395         0.528811   
14  1.855134     2.373158  1.966724     2.386324  1.769256         2.397105   
15  2.354303     2.624658  2.558095     2.623565  2.261542         2.613733   
16  0.364120     0.437347  0.328170     0.495116  0.358532         0.455304   
17  0.251499     0.360473  0.444751     0.374935  0.417615         0.359684   
18  0.236516     0.289094  0.198668     0.304554  0.225824         0.290624   
19  0.309668     0.501987  0.302008     0.488705  0.309096         0.484308   
20  0.305140     0.462510  0.286754     0.455146  0.279598         0.446431   

              
         std  
모델            
0   0.349715  
1   0.319859  
2   0.188028  
3   0.162118  
4   2.605071  
5   0.314087  
6   0.281033  
7   0.358272  
8   0.378310  
9   0.362664  
10  0.291697  



Fold 2 stats:


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.465657  0.293966  0.486433  0.332446  0.459418  0.287960     0.468815   
1   0.537433  0.344585  0.528023  0.347880  0.577739  0.382899     0.523766   
2   0.278368  0.235725  0.307729  0.283691  0.282901  0.217184     0.263564   
3   0.254930  0.207821  0.326728  0.201367  0.288191  0.208499     0.282293   
4   5.965469  3.412544  5.919326  2.859587  5.812719  3.124994     5.864498   
5   0.486555  0.333489  0.462454  0.339853  0.505876  0.355879     0.466452   
6   0.385490  0.262762  0.393822  0.275862  0.360788  0.299259     0.365497   
7   0.658976  0.667819  0.613206  0.454655  0.563369  0.353145     0.614154   
8   0.601213  0.428870  0.733901  0.494154  0.598451  0.377282     0.642897   
9   0.547198  0.387916  0.763959  0.565771  0.520801  0.324518     0.614329   
10  0.492915  0.312040  0.566356  0.433719  0.472791  0.326321     0.499416   
11  0.360581  0.258280  0.377356  0.273197  0.367119  0.235427     0.341264   
12  0.297034  0.217644  0.350865  0.305737  0.312266  0.212914     0.290384   
13  0.564060  0.384941  0.611144  0.452847  0.524846  0.321182     0.555185   
14  2.340178  2.411277  2.328623  1.940373  2.232491  2.321077     2.298224   
15  3.271080  2.709062  3.147495  2.023412  2.949498  2.329527     3.199060   
16  0.333510  0.232741  0.494457  0.417714  0.463450  0.399353     0.348619   
17  0.387442  0.317652  0.471859  0.345232  0.393373  0.290254     0.409954   
18  0.320656  0.196875  0.470608  0.316599  0.301466  0.202526     0.378699   
19  0.500643  0.313829  0.634495  0.392300  0.543816  0.318153     0.551324   
20  0.490877  0.328972  0.585056  0.379860  0.530806  0.334009     0.525900   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.302389     0.456335  0.293525     0.468810  0.299939         0.463843   
1   0.327293     0.552282  0.354559     0.530295  0.352479         0.529271   
2   0.264870     0.276250  0.225297     0.268767  0.250904         0.265304   
3   0.158891     0.263772  0.189484     0.290240  0.178904         0.275659   
4   3.130567     5.811194  3.292513     5.866023  2.907941         5.847238   
5   0.319725     0.492277  0.340784     0.470816  0.330068         0.474749   
6   0.238284     0.361346  0.271782     0.347900  0.233546         0.351260   
7   0.498785     0.602304  0.474328     0.565819  0.379123         0.588537   
8   0.460950     0.586838  0.398941     0.640498  0.402716         0.614913   
9   0.470792     0.530908  0.342877     0.594147  0.453057         0.569271   
10  0.310491     0.481579  0.311568     0.487080  0.324122         0.476743   
11  0.257509     0.362113  0.238824     0.345191  0.250295         0.342268   
12  0.222707     0.299743  0.210687     0.294728  0.232833         0.286748   
13  0.347621     0.540123  0.346813     0.538106  0.331959         0.536113   
14  2.133333     2.277088  2.351283     2.251276  2.099237         2.268146   
15  2.319100     3.110289  2.483655     3.036648  2.155740         3.112351   
16  0.257628     0.357771  0.305359     0.394366  0.325408         0.346747   
17  0.304106     0.383827  0.298516     0.410763  0.298194         0.395991   
18  0.225377     0.309118  0.194175     0.370788  0.220540         0.345478   
19  0.335067     0.520283  0.309715     0.564635  0.337751         0.540500   
20  0.345251     0.499472  0.334968     0.546496  0.345075         0.517207   

              
         std  
모델            
0   0.294508  
1   0.343471  
2   0.244591  
3   0.163532  
4   3.071338  
5   0.324391  
6   0.236278  
7   0.433068  
8   0.418388  
9   0.418964  
10  0.306270  



Fold 3 stats:


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.496268  0.295305  0.475252  0.339685  0.480504  0.330092     0.480032   
1   0.526574  0.323815  0.541840  0.353940  0.533234  0.335673     0.525529   
2   0.271431  0.177815  0.290990  0.240672  0.282898  0.201695     0.272575   
3   0.263651  0.133042  0.356122  0.162760  0.264827  0.135180     0.306353   
4   4.358778  3.387460  4.151679  2.991426  4.246419  3.199355     4.201325   
5   0.442425  0.315533  0.545162  0.346213  0.437419  0.321606     0.484997   
6   0.339335  0.259251  0.405234  0.308018  0.394250  0.234710     0.351158   
7   0.551869  0.351983  0.541740  0.411209  0.606110  0.391052     0.528999   
8   0.489878  0.317974  0.513302  0.449695  0.518257  0.319876     0.477489   
9   0.579707  0.346505  0.602375  0.433953  0.590251  0.339964     0.569426   
10  0.507312  0.314985  0.579261  0.367970  0.507864  0.312453     0.530192   
11  0.299468  0.255572  0.333575  0.281027  0.301966  0.242770     0.293248   
12  0.282810  0.220172  0.383529  0.305721  0.298240  0.231909     0.286561   
13  0.553567  0.300833  0.658678  0.447572  0.521636  0.344198     0.564218   
14  2.218565  2.185698  2.315860  1.687971  2.198116  2.055863     2.251714   
15  2.692246  2.426786  2.943510  1.974078  2.714961  2.196740     2.815172   
16  0.333048  0.279548  0.564956  0.434857  0.541822  0.433027     0.358838   
17  0.371238  0.285077  0.433106  0.299570  0.436442  0.354979     0.368885   
18  0.302493  0.171552  0.392538  0.312255  0.324491  0.181135     0.326605   
19  0.496628  0.314935  0.603335  0.380036  0.519931  0.313483     0.520398   
20  0.496142  0.334380  0.487010  0.306193  0.520807  0.310987     0.482049   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.311770     0.485320  0.309912     0.474849  0.331259         0.479214   
1   0.325644     0.526688  0.323286     0.525359  0.335000         0.524433   
2   0.203077     0.271573  0.186939     0.278126  0.212399         0.272384   
3   0.132204     0.263317  0.122646     0.306690  0.122451         0.291879   
4   3.142672     4.254336  3.295899     4.162632  3.066792         4.174020   
5   0.323528     0.434834  0.316456     0.478834  0.324977         0.464968   
6   0.274653     0.353938  0.238929     0.368952  0.251664         0.347600   
7   0.349088     0.577464  0.346619     0.554149  0.357344         0.549488   
8   0.342029     0.502863  0.305672     0.487983  0.361493         0.480841   
9   0.366434     0.569451  0.320836     0.567549  0.365943         0.561129   
10  0.319714     0.503747  0.311242     0.524980  0.314630         0.516973   
11  0.257646     0.294400  0.246417     0.294500  0.255349         0.287656   
12  0.249347     0.284897  0.215489     0.302481  0.252182         0.277552   
13  0.365926     0.533710  0.305043     0.551307  0.374871         0.538291   
14  1.869345     2.207425  2.095068     2.228833  1.829350         2.216268   
15  2.082448     2.699299  2.272955     2.821629  2.022849         2.777389   
16  0.269472     0.416110  0.307655     0.438560  0.361810         0.362301   
17  0.257711     0.383249  0.269985     0.402148  0.251578         0.367832   
18  0.231841     0.311642  0.170142     0.335499  0.232291         0.317461   
19  0.317609     0.503237  0.308881     0.536003  0.315271         0.513075   
20  0.308865     0.506596  0.313739     0.493596  0.292250         0.491008   

              
         std  
모델            
0   0.315569  
1   0.322504  
2   0.197316  
3   0.117056  
4   3.183420  
5   0.316743  
6   0.254550  
7   0.338556  
8   0.330359  
9   0.343511  
10  0.306520  



Fold 4 stats:


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.571007  0.364296  0.551468  0.348811  0.555821  0.345108     0.556206   
1   0.520608  0.327910  0.525635  0.359967  0.540929  0.329443     0.513827   
2   0.265139  0.187184  0.320748  0.260177  0.288847  0.218801     0.271543   
3   0.281218  0.168524  0.347245  0.195657  0.294871  0.200024     0.305549   
4   5.350283  3.477613  5.349007  2.526641  5.233575  2.957524     5.285088   
5   0.476244  0.344329  0.525065  0.364339  0.484441  0.365573     0.481303   
6   0.443226  0.372334  0.449944  0.402448  0.402187  0.320973     0.437674   
7   0.574280  0.376138  0.580355  0.387403  0.572575  0.371395     0.565187   
8   0.548764  0.369718  0.535236  0.330566  0.584402  0.382630     0.525825   
9   0.472225  0.319034  0.688776  0.458477  0.492828  0.333636     0.565791   
10  0.510812  0.379667  0.656261  0.431472  0.527447  0.314409     0.560779   
11  0.356127  0.263721  0.451327  0.371949  0.325919  0.271232     0.382373   
12  0.320832  0.249203  0.472757  0.323946  0.351835  0.255064     0.365546   
13  0.519673  0.301515  0.633018  0.422572  0.527494  0.323115     0.547019   
14  1.796854  2.007132  1.852472  1.689658  1.792124  1.950082     1.789826   
15  2.625496  2.308265  2.853804  1.993330  2.804904  2.323056     2.731198   
16  0.442435  0.537741  0.737612  0.548337  0.525395  0.487835     0.517053   
17  0.421963  0.267682  0.548969  0.364823  0.436503  0.345650     0.452432   
18  0.287130  0.208329  0.472512  0.440606  0.304140  0.208718     0.355573   
19  0.544125  0.334741  0.632048  0.411406  0.570830  0.354306     0.558003   
20  0.488622  0.304075  0.552250  0.314330  0.504071  0.332252     0.513718   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.344684     0.560680  0.353097     0.549224  0.335493         0.554053   
1   0.337214     0.528147  0.325112     0.521067  0.336824         0.519072   
2   0.219857     0.270279  0.198320     0.283770  0.227532         0.267886   
3   0.157751     0.280883  0.174123     0.311116  0.174798         0.297429   
4   2.955690     5.234257  3.194342     5.284406  2.600984         5.267917   
5   0.346504     0.471532  0.354872     0.477714  0.346506         0.472366   
6   0.378661     0.419617  0.338524     0.404679  0.359398         0.415654   
7   0.326813     0.571465  0.362820     0.563489  0.345591         0.563616   
8   0.334022     0.555648  0.353197     0.547064  0.330005         0.541343   
9   0.354996     0.478820  0.311824     0.574613  0.340089         0.533722   
10  0.377219     0.514441  0.325217     0.571143  0.346101         0.540400   
11  0.311549     0.338769  0.259474     0.366731  0.311128         0.356851   
12  0.288395     0.329487  0.246279     0.379761  0.283369         0.350413   
13  0.322515     0.519963  0.302730     0.557563  0.326258         0.531071   
14  1.825092     1.789635  1.962290     1.795363  1.799974         1.781318   
15  2.084242     2.706921  2.301928     2.813707  2.126976         2.746888   
16  0.399054     0.475918  0.465583     0.572656  0.352679         0.510498   
17  0.289442     0.420722  0.296467     0.460341  0.309032         0.428505   
18  0.274449     0.282288  0.189414     0.361677  0.276935         0.321454   
19  0.351958     0.553019  0.341879     0.562876  0.369249         0.550286   
20  0.283824     0.481003  0.320226     0.507143  0.304725         0.496256   

              
         std  
모델            
0   0.341614  
1   0.329777  
2   0.214480  
3   0.159743  
4   2.870865  
5   0.343601  
6   0.358267  
7   0.333786  
8   0.326499  
9   0.324358  
10  0.344114  



Fold 5 stats:


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.544366  0.335873  0.551949  0.341396  0.516224  0.319958     0.531241   
1   0.478217  0.307140  0.495352  0.324351  0.507611  0.321185     0.477038   
2   0.292416  0.198085  0.282534  0.258803  0.328563  0.218345     0.271581   
3   0.263947  0.206792  0.358711  0.175503  0.273475  0.193592     0.300810   
4   4.952612  3.287213  4.632185  2.813536  4.831942  2.952170     4.771616   
5   0.493817  0.333735  0.493047  0.354156  0.527518  0.320552     0.484136   
6   0.364621  0.344090  0.380055  0.368277  0.323498  0.258235     0.347684   
7   0.524286  0.315532  0.652552  0.425269  0.503380  0.308329     0.561262   
8   0.626701  0.385710  0.703958  0.480532  0.654637  0.418456     0.658352   
9   0.560766  0.435809  0.779014  0.653350  0.605913  0.506891     0.653052   
10  0.502987  0.340441  0.585281  0.407829  0.508115  0.362493     0.528698   
11  0.276812  0.209808  0.392713  0.271435  0.269526  0.218430     0.302909   
12  0.300197  0.192031  0.375002  0.326972  0.298793  0.216815     0.314477   
13  0.584575  0.304591  0.640037  0.428905  0.575050  0.319708     0.583202   
14  2.027631  1.900647  2.115864  1.777676  1.971753  1.765971     2.050433   
15  2.920468  2.272148  3.102292  2.004266  2.927522  2.284083     3.005563   
16  0.441314  0.440640  0.552964  0.530987  0.405422  0.372613     0.416017   
17  0.399934  0.284583  0.482111  0.338671  0.428859  0.304117     0.428350   
18  0.252317  0.188272  0.416368  0.332629  0.267342  0.186943     0.302338   
19  0.497071  0.358812  0.610648  0.406772  0.513630  0.342544     0.520445   
20  0.608564  0.359292  0.601347  0.382323  0.583419  0.302024     0.602643   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.322464     0.524056  0.328019     0.524057  0.322224         0.521621   
1   0.305640     0.486607  0.308893     0.492938  0.311651         0.482594   
2   0.224635     0.302574  0.204739     0.278429  0.235050         0.280482   
3   0.169394     0.262024  0.198821     0.301589  0.160844         0.286796   
4   2.770082     4.862546  2.878822     4.731822  2.777856         4.785759   
5   0.339639     0.507916  0.321405     0.497557  0.331429         0.494885   
6   0.317894     0.318191  0.277161     0.332184  0.270776         0.324205   
7   0.357041     0.503185  0.306049     0.545940  0.353522         0.530367   
8   0.404407     0.637087  0.392011     0.676380  0.417847         0.656391   
9   0.518410     0.568562  0.469425     0.657661  0.528846         0.622266   
10  0.359568     0.503854  0.346412     0.530849  0.366424         0.516482   
11  0.229587     0.269768  0.211111     0.300407  0.231519         0.281005   
12  0.219231     0.295651  0.197962     0.307624  0.232684         0.300515   
13  0.364922     0.576837  0.306867     0.589327  0.356077         0.574028   
14  1.780229     1.996270  1.816141     2.026137  1.721658         2.021507   
15  2.065863     2.922491  2.239486     3.010504  2.092664         2.977026   
16  0.327783     0.393422  0.364457     0.387740  0.350496         0.372428   
17  0.292363     0.412739  0.282595     0.445703  0.293713         0.424178   
18  0.254117     0.254126  0.184554     0.301981  0.247766         0.273214   
19  0.324377     0.484984  0.340151     0.537263  0.325072         0.499096   
20  0.352288     0.593484  0.323696     0.587366  0.328625         0.594432   

              
         std  
모델            
0   0.323130  
1   0.305993  
2   0.217684  
3   0.167386  
4   2.728655  
5   0.327381  
6   0.276080  
7   0.331322  
8   0.395110  
9   0.489475  
10  0.353857  


In [22]:
print("\nFold별 모델 성능 통계 (배터리 용량 Not NaN):")
for i, stats in enumerate(fold_stats_notnan_list, 1):
    print(f"\nFold {i} stats (Not NaN):")
    display(stats)


Fold별 모델 성능 통계 (배터리 용량 Not NaN):

Fold 1 stats (Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.409125  0.392791  0.425636  0.399558  0.463982  0.414564     0.414891   
1   0.569290  0.337927  0.632915  0.332067  0.606895  0.350266     0.591443   
2   0.266028  0.200832  0.294638  0.251503  0.323108  0.248856     0.267521   
3   0.267479  0.191708  0.344847  0.178123  0.278811  0.194058     0.297909   
4   5.150235  2.874002  4.809420  3.026581  4.965147  3.424812     4.979827   
5   0.516363  0.317381  0.555290  0.366674  0.544666  0.348510     0.524332   
6   0.273670  0.211173  0.345091  0.304729  0.304644  0.256168     0.293243   
7   0.609558  0.403119  0.615357  0.398920  0.628414  0.381814     0.600038   
8   0.523480  0.295935  0.494365  0.357236  0.648225  0.753869     0.478945   
9   0.563310  0.354367  0.716780  0.447738  0.689353  0.471592     0.609137   
10  0.593076  0.305468  0.554079  0.323890  0.589870  0.318691     0.570710   
11  0.360227  0.252453  0.406253  0.286797  0.368341  0.228526     0.369049   
12  0.270530  0.207718  0.459394  0.613959  0.255225  0.201862     0.308313   
13  0.541896  0.333463  0.600140  0.355341  0.542778  0.336771     0.550088   
14  1.252454  1.623033  1.410035  1.497026  1.242310  1.658550     1.267189   
15  1.490110  1.849398  1.667220  1.582284  1.554807  1.700778     1.533997   
16  0.435978  0.323031  0.710321  0.583519  0.423678  0.355094     0.509579   
17  0.342182  0.277476  0.393889  0.290905  0.397083  0.757216     0.350863   
18  0.270571  0.169049  0.322749  0.269483  0.272741  0.183324     0.266490   
19  0.498383  0.275631  0.498568  0.406870  0.529359  0.292380     0.480678   
20  0.393983  0.305448  0.400630  0.209154  0.478553  0.255716     0.385705   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.394102     0.436553  0.399165     0.441156  0.404787         0.430479   
1   0.311720     0.585372  0.331538     0.611294  0.323262         0.595113   
2   0.211624     0.285916  0.217327     0.296190  0.231346         0.280381   
3   0.169642     0.270372  0.179669     0.306757  0.163584         0.290975   
4   2.717199     5.057691  3.034905     4.887283  3.104954         4.974934   
5   0.336950     0.529595  0.319581     0.539274  0.336743         0.528907   
6   0.237144     0.276968  0.226180     0.295719  0.234093         0.285422   
7   0.378838     0.618457  0.370312     0.613486  0.368595         0.605447   
8   0.331425     0.584512  0.434841     0.545231  0.454527         0.531305   
9   0.364044     0.623380  0.398971     0.658081  0.399864         0.625673   
10  0.285317     0.587665  0.306033     0.564550  0.293355         0.572738   
11  0.263364     0.362074  0.231825     0.371120  0.243747         0.365483   
12  0.327929     0.255731  0.191053     0.326705  0.349368         0.278743   
13  0.318022     0.539828  0.319958     0.550401  0.318404         0.539719   
14  1.552080     1.237909  1.636381     1.263458  1.559560         1.251742   
15  1.683356     1.503383  1.770013     1.579645  1.621989         1.535069   
16  0.370908     0.423151  0.307435     0.502935  0.373690         0.463569   
17  0.265256     0.364575  0.478020     0.375826  0.452641         0.359827   
18  0.183570     0.267050  0.163767     0.269383  0.185031         0.261338   
19  0.306647     0.509781  0.285462     0.491740  0.318167         0.488480   
20  0.250884     0.433647  0.270585     0.430610  0.217202         0.416654   

              
         std  
모델            
0   0.398017  
1   0.313251  
2   0.213742  
3   0.162818  
4   2.901892  
5   0.325756  
6   0.217487  
7   0.369793  
8   0.370063  
9   0.364791  
10  0.287683  



Fold 2 stats (Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.620729  0.325521  0.652613  0.464729  0.648748  0.310641     0.628148   
1   0.561106  0.349595  0.610262  0.380088  0.628093  0.411327     0.578135   
2   0.270700  0.224685  0.313295  0.286059  0.277103  0.207934     0.262943   
3   0.277874  0.214716  0.347982  0.214843  0.310838  0.221643     0.305818   
4   5.931260  3.521497  5.740126  3.054210  5.752757  3.278270     5.731827   
5   0.483507  0.323805  0.466281  0.362463  0.506647  0.373528     0.469676   
6   0.352081  0.273519  0.362146  0.311505  0.333321  0.288813     0.322953   
7   0.654572  0.596365  0.584518  0.494210  0.608690  0.376172     0.596375   
8   0.527557  0.386886  0.635614  0.409715  0.545593  0.360685     0.551191   
9   0.532540  0.404590  0.743158  0.573489  0.494304  0.323833     0.590538   
10  0.487476  0.291027  0.512224  0.370195  0.473568  0.322420     0.491745   
11  0.387206  0.295456  0.406854  0.266594  0.392380  0.255029     0.362816   
12  0.402066  0.236357  0.468849  0.372820  0.432757  0.247992     0.388753   
13  0.550892  0.295030  0.605640  0.459249  0.538202  0.299921     0.544656   
14  1.247169  1.880626  1.315885  1.440921  1.051311  1.480178     1.230668   
15  1.553147  1.400588  1.625684  1.086534  1.420069  1.322585     1.571383   
16  0.295918  0.219914  0.574446  0.462350  0.423200  0.364691     0.364929   
17  0.399281  0.324946  0.476549  0.357660  0.402113  0.297411     0.416531   
18  0.317493  0.184048  0.422261  0.270680  0.296033  0.197529     0.352026   
19  0.483573  0.309180  0.622198  0.392896  0.511283  0.309693     0.536394   
20  0.530626  0.448390  0.762376  0.500649  0.576169  0.438718     0.625203   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.398596     0.626215  0.330336     0.650681  0.372251         0.635015   
1   0.338623     0.585141  0.370377     0.588479  0.378042         0.575205   
2   0.264958     0.269176  0.215886     0.269093  0.251015         0.263561   
3   0.162438     0.284105  0.194661     0.309220  0.190159         0.296300   
4   3.306522     5.738142  3.443690     5.746441  3.075365         5.738804   
5   0.331340     0.490707  0.344301     0.475185  0.349958         0.476867   
6   0.254143     0.329166  0.274263     0.311570  0.225086         0.313314   
7   0.515976     0.619480  0.472950     0.583369  0.412321         0.598972   
8   0.398697     0.520559  0.366896     0.558954  0.344634         0.533093   
9   0.475452     0.508387  0.348706     0.563131  0.459112         0.539617   
10  0.302692     0.479481  0.299500     0.476105  0.319393         0.479315   
11  0.273568     0.388473  0.263262     0.363887  0.261364         0.361865   
12  0.247562     0.413373  0.235308     0.412301  0.256427         0.395134   
13  0.334118     0.542929  0.292812     0.541480  0.335005         0.534986   
14  1.593790     1.139339  1.678548     1.139066  1.419819         1.158461   
15  1.232012     1.486608  1.348452     1.501987  1.204439         1.514735   
16  0.289597     0.318843  0.278345     0.413232  0.351471         0.348179   
17  0.314953     0.393552  0.306312     0.415605  0.309382         0.402565   
18  0.195876     0.304501  0.184974     0.343088  0.193541         0.325787   
19  0.330897     0.495149  0.303101     0.540585  0.334132         0.518295   
20  0.474207     0.553398  0.433213     0.663323  0.442855         0.605876   

              
         std  
모델            
0   0.362916  
1   0.361465  
2   0.241748  
3   0.168203  
4   3.231469  
5   0.336910  
6   0.234644  
7   0.452942  
8   0.367151  
9   0.425899  
10  0.299865  



Fold 3 stats (Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.597612  0.272554  0.528289  0.430483  0.510691  0.337130     0.562951   
1   0.528414  0.313328  0.534491  0.340122  0.549443  0.330774     0.517264   
2   0.275421  0.175243  0.288163  0.250428  0.293957  0.195254     0.272646   
3   0.274736  0.137414  0.356675  0.169648  0.279479  0.130868     0.311683   
4   4.665268  3.368104  4.547459  2.945372  4.603994  2.947124     4.568305   
5   0.457401  0.321195  0.547873  0.319464  0.436426  0.320544     0.497255   
6   0.313405  0.237771  0.356358  0.244109  0.394570  0.243999     0.310211   
7   0.553300  0.360736  0.554691  0.423584  0.612676  0.396473     0.534410   
8   0.486687  0.313861  0.524439  0.463572  0.526627  0.328394     0.476893   
9   0.599785  0.339972  0.624026  0.430809  0.676515  0.337386     0.581427   
10  0.496266  0.324205  0.562242  0.338065  0.501134  0.324000     0.522525   
11  0.343548  0.280687  0.367236  0.324988  0.342137  0.256726     0.337999   
12  0.187534  0.155008  0.504079  0.333030  0.248476  0.225740     0.304752   
13  0.518785  0.279655  0.664542  0.466330  0.505111  0.322483     0.557601   
14  1.234710  1.915892  1.468920  1.470825  1.191048  1.655600     1.327826   
15  1.327028  1.427586  1.606541  1.385205  1.361042  1.294801     1.462136   
16  0.313921  0.269042  0.618065  0.464922  0.445838  0.283537     0.350694   
17  0.385782  0.296149  0.438999  0.295563  0.460705  0.366044     0.374564   
18  0.301910  0.175695  0.390786  0.315210  0.318181  0.185828     0.324149   
19  0.507720  0.310503  0.605468  0.397989  0.539837  0.319057     0.531653   
20  0.476866  0.344777  0.519487  0.330182  0.515537  0.299746     0.482404   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.333202     0.550088  0.300373     0.515426  0.377987         0.542822   
1   0.311132     0.536125  0.311006     0.524029  0.324727         0.523666   
2   0.211212     0.278309  0.182803     0.282569  0.215852         0.275080   
3   0.139557     0.276320  0.120550     0.314842  0.120644         0.300948   
4   3.076753     4.621318  3.131212     4.567646  2.881508         4.574815   
5   0.310383     0.441621  0.317397     0.480692  0.306192         0.471109   
6   0.224836     0.333421  0.236351     0.336071  0.229556         0.312921   
7   0.361832     0.582670  0.352348     0.563711  0.370791         0.555949   
8   0.353374     0.506306  0.307462     0.490279  0.377915         0.481272   
9   0.358833     0.615461  0.328560     0.609351  0.383487         0.590851   
10  0.303737     0.494181  0.322256     0.516385  0.303990         0.508908   
11  0.283630     0.339033  0.266008     0.339142  0.276178         0.332219   
12  0.242987     0.203876  0.190609     0.340581  0.257529         0.266315   
13  0.368440     0.509445  0.292383     0.548937  0.375905         0.529572   
14  1.642357     1.212221  1.774208     1.284935  1.526857         1.254105   
15  1.291508     1.336640  1.328163     1.470724  1.284982         1.420915   
16  0.271036     0.368539  0.231809     0.389870  0.318087         0.319215   
17  0.263825     0.400358  0.278640     0.413290  0.251359         0.376834   
18  0.235589     0.307924  0.175004     0.328626  0.238507         0.312841   
19  0.320815     0.519440  0.311839     0.549719  0.320387         0.528625   
20  0.329417     0.496201  0.305761     0.494108  0.290530         0.481495   

              
         std  
모델            
0   0.332360  
1   0.308398  
2   0.201978  
3   0.117330  
4   3.017857  
5   0.307563  
6   0.231070  
7   0.350247  
8   0.341721  
9   0.353406  
10  0.301983  



Fold 4 stats (Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.613556  0.303054  0.617331  0.327251  0.589984  0.321269     0.603489   
1   0.529987  0.348267  0.528994  0.392065  0.556414  0.348583     0.518068   
2   0.233357  0.170593  0.296874  0.269557  0.261306  0.221272     0.232864   
3   0.298010  0.181177  0.365700  0.209131  0.324192  0.216536     0.323626   
4   5.700380  3.331838  5.689433  2.228701  5.373736  2.975667     5.634334   
5   0.444278  0.322054  0.488329  0.337995  0.466179  0.361456     0.438616   
6   0.393228  0.336397  0.354358  0.375667  0.353186  0.262184     0.361515   
7   0.636749  0.396666  0.568030  0.389709  0.606145  0.390947     0.595987   
8   0.544392  0.378009  0.501206  0.312907  0.598491  0.404317     0.503748   
9   0.408261  0.322725  0.746604  0.470023  0.445781  0.358141     0.559309   
10  0.453657  0.304050  0.598574  0.348574  0.508047  0.323501     0.507463   
11  0.373324  0.271859  0.481869  0.383806  0.341569  0.277638     0.404296   
12  0.300652  0.217164  0.526427  0.321236  0.329280  0.290562     0.354521   
13  0.478415  0.291025  0.625928  0.420002  0.484617  0.303359     0.520828   
14  0.740566  1.410659  0.805210  1.134961  0.668237  1.185539     0.728094   
15  1.506140  1.494135  1.658787  1.291446  1.582677  1.446817     1.569292   
16  0.332770  0.476162  0.826146  0.585136  0.415250  0.337245     0.516798   
17  0.428804  0.281861  0.506841  0.324957  0.460580  0.371464     0.431532   
18  0.280670  0.182139  0.461748  0.455627  0.287374  0.189876     0.351058   
19  0.528833  0.307740  0.628905  0.444169  0.555226  0.333603     0.540789   
20  0.604267  0.307500  0.725819  0.394485  0.580494  0.296554     0.654644   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.303825     0.589798  0.321825     0.603657  0.280354         0.593951   
1   0.362954     0.539319  0.345824     0.528752  0.360295         0.526464   
2   0.214119     0.236911  0.188894     0.247600  0.226775         0.227590   
3   0.161187     0.303118  0.185000     0.336727  0.186707         0.319643   
4   2.715794     5.476485  3.135420     5.531584  2.461689         5.547468   
5   0.320466     0.443306  0.340770     0.441287  0.320970         0.435818   
6   0.338650     0.369898  0.289808     0.318028  0.319144         0.340826   
7   0.332474     0.619030  0.382725     0.582506  0.349894         0.598326   
8   0.342390     0.558562  0.372048     0.534825  0.343881         0.530609   
9   0.378449     0.420698  0.327098     0.579515  0.371765         0.513660   
10  0.312588     0.477029  0.304031     0.535969  0.317263         0.501314   
11  0.318808     0.355327  0.266227     0.386923  0.322651         0.375790   
12  0.273225     0.310889  0.250686     0.361895  0.294987         0.326555   
13  0.325451     0.477182  0.290406     0.531869  0.329621         0.497458   
14  1.261813     0.703541  1.293557     0.693628  1.154817         0.693302   
15  1.373527     1.531509  1.461297     1.596352  1.372203         1.559920   
16  0.395654     0.362386  0.370128     0.580854  0.316103         0.477489   
17  0.283790     0.434259  0.317488     0.448513  0.313639         0.420872   
18  0.278170     0.281886  0.180811     0.349159  0.266812         0.319830   
19  0.362269     0.540006  0.317198     0.549312  0.376135         0.534613   
20  0.288454     0.571503  0.324893     0.616378  0.269514         0.612341   

              
         std  
모델            
0   0.302543  
1   0.352528  
2   0.208407  
3   0.166488  
4   2.722672  
5   0.318003  
6   0.316961  
7   0.341390  
8   0.342880  
9   0.350506  
10  0.307316  



Fold 5 stats (Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.418333  0.280097  0.501238  0.320412  0.389153  0.227094     0.439209   
1   0.466566  0.319753  0.487602  0.345581  0.498385  0.338043     0.470727   
2   0.312283  0.208870  0.255195  0.270299  0.356393  0.243217     0.278446   
3   0.263297  0.181016  0.357104  0.165529  0.271876  0.175688     0.303125   
4   4.835967  3.496963  4.724244  2.906039  4.879543  3.099691     4.755859   
5   0.482048  0.333376  0.472680  0.344687  0.528596  0.315129     0.464854   
6   0.311533  0.249416  0.377549  0.392943  0.315567  0.255092     0.313920   
7   0.547831  0.313893  0.640930  0.436707  0.532112  0.299917     0.568067   
8   0.697551  0.413104  0.747038  0.493465  0.738241  0.429917     0.716823   
9   0.629953  0.440316  0.736375  0.555045  0.632280  0.582437     0.669270   
10  0.492778  0.336455  0.583602  0.398483  0.500265  0.367199     0.527701   
11  0.299540  0.250101  0.418857  0.286095  0.302420  0.251175     0.331357   
12  0.360115  0.253888  0.394842  0.418748  0.333429  0.272717     0.340141   
13  0.589162  0.306153  0.649468  0.450919  0.577038  0.326854     0.595741   
14  1.189813  1.481207  1.233010  1.423956  1.141048  1.368288     1.180093   
15  1.609035  1.369485  1.752967  1.406899  1.558452  1.533272     1.671258   
16  0.382456  0.425938  0.717315  0.568326  0.365731  0.394060     0.446905   
17  0.419132  0.292918  0.478738  0.349239  0.457583  0.309293     0.437387   
18  0.255312  0.183709  0.420192  0.319378  0.259912  0.178466     0.301512   
19  0.482271  0.335791  0.663593  0.413162  0.482984  0.344035     0.536296   
20  0.570935  0.300221  0.559198  0.304756  0.595865  0.261966     0.565067   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.219621     0.392885  0.255276     0.422814  0.237195         0.407603   
1   0.318817     0.475211  0.322595     0.482193  0.329680         0.473400   
2   0.232622     0.321920  0.225041     0.290700  0.249017         0.294228   
3   0.147684     0.257622  0.178629     0.297928  0.139085         0.285196   
4   2.918361     4.823068  3.045283     4.801612  2.887510         4.790127   
5   0.338515     0.503223  0.313905     0.489349  0.320236         0.485516   
6   0.301797     0.281987  0.240211     0.323450  0.269689         0.297327   
7   0.361796     0.537377  0.291200     0.566295  0.351764         0.551809   
8   0.418353     0.713701  0.408652     0.738344  0.427911         0.722898   
9   0.479101     0.622561  0.507709     0.668038  0.527735         0.651034   
10  0.346325     0.494633  0.347027     0.529607  0.360863         0.512597   
11  0.243689     0.298469  0.245836     0.335179  0.242181         0.312591   
12  0.231491     0.338097  0.262437     0.323514  0.256363         0.320146   
13  0.372085     0.580429  0.314478     0.595234  0.370606         0.581933   
14  1.387493     1.160402  1.414161     1.161064  1.324583         1.163111   
15  1.357833     1.581223  1.444945     1.648335  1.433696         1.629429   
16  0.363617     0.360005  0.367300     0.431840  0.395341         0.379474   
17  0.303799     0.437564  0.287295     0.457647  0.304575         0.439195   
18  0.246155     0.251154  0.179965     0.294900  0.237854         0.267876   
19  0.319016     0.476470  0.338690     0.544254  0.332129         0.507594   
20  0.255666     0.583400  0.276238     0.577531  0.248750         0.575333   

              
         std  
모델            
0   0.230724  
1   0.319814  
2   0.232133  
3   0.143829  
4   2.866258  
5   0.318226  
6   0.258238  
7   0.328732  
8   0.405722  
9   0.493845  
10  0.348287  


In [23]:
print("\nFold별 모델 성능 통계 (배터리 용량 NaN):")
for i, stats in enumerate(fold_stats_isnan_list, 1):
    print(f"\nFold {i} stats (NaN):")
    display(stats)


Fold별 모델 성능 통계 (배터리 용량 NaN):

Fold 1 stats (NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.561283  0.345272  0.541030  0.325630  0.531474  0.337306     0.545861   
1   0.561257  0.349087  0.537073  0.327576  0.583456  0.341756     0.542560   
2   0.272971  0.168456  0.223622  0.191315  0.268634  0.144980     0.237251   
3   0.285272  0.190061  0.342008  0.155033  0.303126  0.188682     0.313640   
4   4.928991  2.130038  5.644177  1.522510  5.822626  2.034902     5.286584   
5   0.536448  0.290988  0.488127  0.345980  0.518773  0.297681     0.512288   
6   0.500412  0.337722  0.433528  0.310323  0.496551  0.343668     0.445300   
7   0.405739  0.327388  0.658438  0.308601  0.436589  0.319293     0.468715   
8   0.613193  0.483798  0.538022  0.350067  0.533230  0.419959     0.547463   
9   0.505799  0.305867  0.544989  0.400765  0.485156  0.359083     0.476660   
10  0.747128  0.209919  0.994785  0.281651  0.829807  0.281757     0.870957   
11  0.272784  0.177280  0.285575  0.252957  0.275669  0.173444     0.251289   
12  0.295589  0.207035  0.377805  0.271651  0.319751  0.235206     0.313927   
13  0.453031  0.332809  0.520603  0.401715  0.461690  0.290774     0.486314   
14  3.749065  1.745103  3.747718  0.805077  3.534867  1.314537     3.748392   
15  5.494667  2.232883  5.101628  0.844313  5.420042  1.816192     5.298147   
16  0.413573  0.389144  0.381526  0.321661  0.610064  0.431065     0.370374   
17  0.342688  0.242344  0.414455  0.271628  0.345348  0.221907     0.368936   
18  0.379791  0.345659  0.539926  0.380491  0.390842  0.280295     0.431547   
19  0.554536  0.528609  0.504349  0.275918  0.465887  0.313769     0.490052   
20  0.454517  0.319515  0.447095  0.349063  0.499754  0.291657     0.441201   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.331625     0.543329  0.340763     0.531749  0.331668         0.539153   
1   0.335941     0.569372  0.341130     0.553778  0.330817         0.554423   
2   0.167015     0.268902  0.138399     0.228725  0.159063         0.242627   
3   0.156709     0.290075  0.184554     0.317470  0.161864         0.306645   
4   1.725561     5.375808  2.028291     5.733401  1.638275         5.465265   
5   0.295761     0.524038  0.296483     0.499878  0.299250         0.512068   
6   0.331925     0.490714  0.338189     0.423940  0.336091         0.448700   
7   0.257466     0.388552  0.343385     0.443837  0.305063         0.419054   
8   0.421306     0.567866  0.448841     0.524805  0.366490         0.536376   
9   0.355105     0.494058  0.314653     0.453191  0.373746         0.463254   
10  0.207218     0.788468  0.241891     0.912296  0.230701         0.857240   
11  0.211484     0.266547  0.174216     0.252473  0.217652         0.255193   
12  0.228713     0.305737  0.209495     0.317640  0.241056         0.304753   
13  0.344381     0.457360  0.303165     0.482881  0.337740         0.472595   
14  1.164425     3.641966  1.475478     3.641292  0.966043         3.677217   
15  1.486463     5.457354  1.997752     5.260835  1.279204         5.338779   
16  0.329174     0.482300  0.393230     0.470358  0.314152         0.429130   
17  0.177485     0.340594  0.234858     0.370618  0.177257         0.358987   
18  0.367232     0.381678  0.294850     0.452273  0.316096         0.413626   
19  0.329353     0.473842  0.363450     0.477742  0.282090         0.469242   
20  0.325366     0.474166  0.294775     0.465054  0.302516         0.458457   

              
         std  
모델            
0   0.333772  
1   0.333173  
2   0.149241  
3   0.163331  
4   1.767168  
5   0.290400  
6   0.333155  
7   0.278313  
8   0.418426  
9   0.343593  
10  0.214971  



Fold 2 stats (NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.428861  0.276301  0.447000  0.283799  0.414493  0.265711     0.431007   
1   0.509955  0.339734  0.432567  0.280477  0.519292  0.341364     0.460659   
2   0.295099  0.262986  0.295585  0.284725  0.295551  0.240772     0.264917   
3   0.202059  0.184596  0.277749  0.159806  0.236005  0.167336     0.228082   
4   6.068095  3.324411  6.456925  2.292162  5.992607  2.841001     6.262510   
5   0.492779  0.359548  0.454641  0.295443  0.504301  0.324478     0.459871   
6   0.450549  0.233624  0.455507  0.179936  0.414277  0.319761     0.448346   
7   0.669985  0.842848  0.684929  0.339450  0.450065  0.264934     0.658603   
8   0.917933  0.476252  1.156534  0.618764  0.825741  0.380218     1.037233   
9   0.570518  0.367861  0.797052  0.565008  0.562955  0.328656     0.652179   
10  0.530984  0.451108  0.945285  0.646686  0.467354  0.371058     0.553113   
11  0.319756  0.184979  0.332126  0.281504  0.328385  0.199637     0.308217   
12  0.252298  0.194703  0.300612  0.260250  0.260946  0.174622     0.248486   
13  0.615910  0.638093  0.632817  0.440432  0.472254  0.401145     0.596641   
14  4.438757  1.881423  4.273081  1.113286  4.500355  1.929703     4.347931   
15  5.522165  2.335218  5.141592  0.910567  4.953577  1.773606     5.331878   
16  0.408694  0.243819  0.334480  0.248119  0.543951  0.458338     0.315999   
17  0.249324  0.177622  0.417135  0.136281  0.291408  0.171786     0.333230   
18  0.339925  0.272780  0.765086  0.420944  0.334554  0.238559     0.541162   
19  0.600215  0.336065  0.706233  0.397964  0.733587  0.312600     0.638413   
20  0.479022  0.288113  0.532171  0.322631  0.517276  0.299375     0.496283   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.265175     0.416024  0.271801     0.425654  0.265972         0.423227   
1   0.304532     0.514143  0.334511     0.462760  0.310041         0.475954   
2   0.270907     0.291685  0.249206     0.268053  0.256564         0.269107   
3   0.138901     0.216917  0.171882     0.246501  0.144138         0.228097   
4   2.722494     6.030351  3.029550     6.224766  2.515179         6.172542   
5   0.301363     0.495483  0.340792     0.461895  0.292115         0.470425   
6   0.182617     0.424013  0.262651     0.418647  0.239454         0.425153   
7   0.465903     0.559363  0.490536     0.521943  0.286582         0.562449   
8   0.522364     0.871837  0.424412     0.991137  0.463025         0.966736   
9   0.471822     0.566737  0.338299     0.643491  0.449376         0.616448   
10  0.374325     0.496267  0.405121     0.563904  0.364096         0.458740   
11  0.231294     0.321695  0.192801     0.316525  0.233699         0.312218   
12  0.199236     0.251345  0.180772     0.244651  0.204668         0.240584   
13  0.405770     0.529078  0.520481     0.524820  0.330017         0.540551   
14  1.422370     4.461568  1.873022     4.386718  1.431248         4.398739   
15  1.537730     5.237871  1.967535     5.047584  1.285526         5.205778   
16  0.178207     0.435628  0.346196     0.356634  0.268439         0.343883   
17  0.103157     0.270366  0.158410     0.354272  0.093478         0.319289   
18  0.322637     0.337239  0.251965     0.539508  0.301384         0.465418   
19  0.360839     0.666901  0.320310     0.704931  0.338451         0.670026   
20  0.295378     0.483389  0.302487     0.511653  0.306287         0.490762   

              
         std  
모델            
0   0.263439  
1   0.316180  
2   0.256409  
3   0.144585  
4   2.734310  
5   0.304134  
6   0.227475  
7   0.391477  
8   0.461312  
9   0.413045  
10  0.365481  



Fold 3 stats (NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.478573  0.297582  0.465992  0.324625  0.475233  0.331317     0.465554   
1   0.523677  0.343111  0.553410  0.378145  0.507714  0.345281     0.538543   
2   0.265078  0.185022  0.295493  0.228861  0.265287  0.214130     0.272462   
3   0.249207  0.127754  0.355402  0.155925  0.245734  0.140307     0.299408   
4   3.711743  3.536860  3.316145  3.086257  3.491539  3.749515     3.426590   
5   0.419701  0.310956  0.541050  0.389180  0.438925  0.328889     0.466399   
6   0.382550  0.292546  0.486693  0.385300  0.393716  0.224241     0.419402   
7   0.537564  0.279577  0.412234  0.248507  0.540449  0.364600     0.474899   
8   0.501774  0.348449  0.471790  0.411527  0.487062  0.298508     0.479710   
9   0.541559  0.364390  0.561238  0.448109  0.426350  0.286178     0.546624   
10  0.569907  0.259441  0.675704  0.514180  0.546001  0.244694     0.573639   
11  0.219472  0.180713  0.272485  0.163766  0.229063  0.199437     0.212033   
12  0.323390  0.232211  0.332183  0.281062  0.319436  0.233315     0.278813   
13  0.701972  0.350941  0.633660  0.369674  0.592142  0.430705     0.592452   
14  3.858324  1.538251  3.727427  0.899426  3.876564  1.490737     3.791529   
15  4.593800  2.255157  4.805717  0.804526  4.600778  1.760884     4.699758   
16  0.382231  0.306267  0.428388  0.315706  0.788638  0.626030     0.379779   
17  0.264583  0.156826  0.389895  0.343436  0.258511  0.190960     0.327239   
18  0.306456  0.148226  0.404450  0.307175  0.367403  0.145896     0.343305   
19  0.436903  0.344681  0.591846  0.275638  0.412743  0.267265     0.459797   
20  0.500639  0.334724  0.479431  0.302772  0.522037  0.316001     0.481966   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.308392     0.474011  0.312505     0.467764  0.325291         0.468107   
1   0.350331     0.511828  0.344642     0.527452  0.354137         0.525641   
2   0.193343     0.260847  0.196382     0.271050  0.210653         0.268091   
3   0.123749     0.246375  0.125132     0.296068  0.125831         0.280060   
4   3.322393     3.479596  3.688802     3.307603  3.442178         3.327897   
5   0.347290     0.424537  0.320348     0.476014  0.357179         0.455650   
6   0.337168     0.388133  0.245071     0.423753  0.282031         0.405397   
7   0.193179     0.525410  0.312092     0.458534  0.164646         0.484877   
8   0.311641     0.490029  0.313304     0.479426  0.308338         0.479235   
9   0.388898     0.482033  0.293780     0.488125  0.324382         0.504655   
10  0.412015     0.557954  0.243742     0.573685  0.380836         0.562672   
11  0.179849     0.213399  0.184270     0.213484  0.191570         0.206783   
12  0.253854     0.319406  0.217815     0.286253  0.250531         0.282338   
13  0.366202     0.637240  0.345773     0.561421  0.383323         0.575493   
14  1.018949     3.866098  1.444958     3.801996  1.039839         3.819874   
15  1.387605     4.597289  1.930859     4.703247  1.170869         4.666765   
16  0.270861     0.538434  0.431266     0.563764  0.439736         0.473094   
17  0.215849     0.257784  0.152648     0.320440  0.252115         0.301821   
18  0.215227     0.336929  0.137320     0.382237  0.188107         0.348878   
19  0.304617     0.415988  0.288137     0.462150  0.286312         0.429345   
20  0.306796     0.509021  0.318053     0.493477  0.295091         0.493228   

              
         std  
모델            
0   0.313993  
1   0.346974  
2   0.193372  
3   0.117436  
4   3.538938  
5   0.335502  
6   0.286020  
7   0.197192  
8   0.299170  
9   0.325048  
10  0.341513  



Fold 4 stats (NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.564358  0.374599  0.541177  0.353393  0.550483  0.350777     0.548818   
1   0.501617  0.285470  0.518832  0.288638  0.509572  0.288424     0.505240   
2   0.320145  0.204748  0.362069  0.242598  0.336515  0.210112     0.338488   
3   0.254003  0.144585  0.317335  0.170850  0.247352  0.162350     0.276252   
4   3.249703  4.107502  3.306455  3.589463  4.392613  3.120453     3.189615   
5   0.541509  0.384802  0.600066  0.410346  0.521726  0.378868     0.568457   
6   0.496926  0.406954  0.552610  0.411703  0.454817  0.371958     0.519474   
7   0.434644  0.289224  0.607904  0.392618  0.497538  0.321603     0.496338   
8   0.573358  0.341227  0.726655  0.383390  0.505153  0.227145     0.650007   
9   0.562842  0.296973  0.606853  0.438193  0.559476  0.289697     0.574974   
10  0.719256  0.539854  0.866651  0.620008  0.598200  0.275939     0.755225   
11  0.316499  0.245058  0.380946  0.340629  0.289858  0.258167     0.331856   
12  0.325316  0.257146  0.460831  0.325892  0.356848  0.248808     0.367996   
13  0.729140  0.274340  0.669015  0.451110  0.745177  0.344525     0.679992   
14  3.557333  1.576043  3.597907  0.744526  3.665268  1.475225     3.559381   
15  4.631011  2.161763  4.994878  0.950180  4.994726  1.966399     4.812944   
16  0.637395  0.592854  0.580218  0.443441  0.721208  0.639942     0.517506   
17  0.391666  0.198477  0.735538  0.475832  0.329877  0.163690     0.544991   
18  0.311759  0.293909  0.513549  0.388530  0.368058  0.266472     0.372787   
19  0.599344  0.424063  0.643396  0.271120  0.627180  0.426899     0.620166   
20  0.468105  0.301321  0.521456  0.291055  0.490511  0.338587     0.488715   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.352233     0.556131  0.359883     0.540719  0.344456         0.547819   
1   0.282041     0.505524  0.281279     0.505503  0.287097         0.504102   
2   0.217485     0.328030  0.204585     0.346373  0.219124         0.337630   
3   0.150119     0.244848  0.151029     0.269610  0.147266         0.261428   
4   3.904958     3.780888  3.630253     3.801340  3.317428         3.590614   
5   0.386937     0.529161  0.383039     0.552086  0.390205         0.546986   
6   0.407908     0.473018  0.382429     0.497749  0.382335         0.496025   
7   0.312348     0.465142  0.296923     0.520980  0.342344         0.486030   
8   0.266784     0.539255  0.237216     0.615904  0.243527         0.601722   
9   0.326605     0.561159  0.274575     0.567669  0.297070         0.562143   
10  0.519590     0.650883  0.371485     0.699427  0.421811         0.682945   
11  0.294675     0.300614  0.244568     0.320202  0.284066         0.313208   
12  0.293711     0.333619  0.247137     0.383731  0.283013         0.355715   
13  0.281869     0.737158  0.278758     0.688010  0.285135         0.701720   
14  1.094910     3.599791  1.496335     3.631588  0.994144         3.594677   
15  1.405163     4.812868  2.023668     4.994802  1.344978         4.873538   
16  0.412618     0.677752  0.550646     0.558082  0.415983         0.569180   
17  0.306777     0.360772  0.171315     0.512719  0.292858         0.462305   
18  0.267803     0.283819  0.225844     0.409402  0.317420         0.327645   
19  0.313383     0.600014  0.426626     0.611856  0.348996         0.606883   
20  0.277895     0.464947  0.319364     0.487762  0.308482         0.475661   

              
         std  
모델            
0   0.349079  
1   0.281700  
2   0.210666  
3   0.143678  
4   3.601098  
5   0.387149  
6   0.387783  
7   0.311845  
8   0.219085  
9   0.288141  
10  0.435541  



Fold 5 stats (NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.581955  0.344066  0.567073  0.348693  0.554123  0.335190     0.558690   
1   0.499352  0.285265  0.509411  0.285166  0.524347  0.291201     0.488486   
2   0.260040  0.178133  0.327087  0.236999  0.283211  0.164548     0.260394   
3   0.265127  0.250718  0.361626  0.195581  0.276378  0.226078     0.296609   
4   5.652486  1.654152  4.079830  2.454338  4.546336  2.156130     4.866158   
5   0.517844  0.340331  0.534629  0.376831  0.525318  0.338242     0.523505   
6   0.460181  0.461719  0.384566  0.328852  0.337772  0.269878     0.408460   
7   0.332009  0.282094  0.747471  0.332033  0.268742  0.298404     0.505685   
8   0.476666  0.273845  0.612729  0.452516  0.477593  0.340220     0.534531   
9   0.447553  0.413231  0.848787  0.798100  0.562767  0.358992     0.626513   
10  0.593592  0.386003  0.600183  0.514939  0.577780  0.331151     0.537545   
11  0.249406  0.146545  0.361186  0.253237  0.229859  0.165993     0.268604   
12  0.277728  0.160097  0.367562  0.289415  0.285805  0.193087     0.304853   
13  0.558580  0.307456  0.586598  0.281151  0.563789  0.288444     0.512148   
14  3.812549  1.406193  3.996726  0.583256  3.741514  1.080980     3.904638   
15  4.863332  1.930321  5.101291  0.526712  4.955774  1.600376     4.982312   
16  0.559032  0.454548  0.224264  0.202663  0.484805  0.318131     0.354243   
17  0.257023  0.159169  0.507221  0.261258  0.215028  0.139656     0.361068   
18  0.229688  0.231172  0.387478  0.442455  0.323476  0.247472     0.308583   
19  0.550517  0.438926  0.419461  0.325658  0.624298  0.322217     0.463204   
20  0.616717  0.372590  0.610479  0.398763  0.580723  0.311958     0.610785   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.344137     0.563177  0.338910     0.554252  0.339460         0.555626   
1   0.283453     0.507279  0.284857     0.512428  0.278633         0.499271   
2   0.214828     0.271049  0.165633     0.258432  0.213360         0.258083   
3   0.206101     0.270013  0.234558     0.308232  0.197086         0.289699   
4   1.936639     5.099411  1.844599     4.313083  2.291961         4.759551   
5   0.345762     0.517498  0.342920     0.514316  0.359731         0.514012   
6   0.344532     0.383358  0.330338     0.347904  0.279025         0.372586   
7   0.340581     0.223953  0.304164     0.379709  0.353073         0.355261   
8   0.353106     0.474846  0.304888     0.545161  0.374107         0.515552   
9   0.587970     0.480199  0.394115     0.640682  0.542648         0.575192   
10  0.491059     0.585686  0.352649     0.541867  0.440154         0.550957   
11  0.209761     0.235158  0.156292     0.258477  0.213986         0.242917   
12  0.215826     0.279733  0.167813     0.301665  0.225350         0.293153   
13  0.326557     0.556486  0.270911     0.555852  0.269398         0.529231   
14  0.863786     3.777031  1.187448     3.869120  0.721231         3.850263   
15  1.127477     4.909553  1.652687     5.028533  0.974485         4.973466   
16  0.235313     0.460256  0.356564     0.299539  0.218000         0.358335   
17  0.184759     0.227936  0.155434     0.356783  0.182619         0.312388   
18  0.325513     0.276582  0.227348     0.355477  0.325161         0.313547   
19  0.346360     0.515727  0.353500     0.512018  0.305884         0.468411   
20  0.371187     0.595669  0.335140     0.589497  0.345223         0.598570   

              
         std  
모델            
0   0.340259  
1   0.282068  
2   0.193957  
3   0.206440  
4   2.005988  
5   0.351575  
6   0.306530  
7   0.326870  
8   0.340958  
9   0.489993  
10  0.425088  


In [27]:
# 'mean' 열만 필터링
mean_columns = [col for col in model_stats_mean.columns if col[1] == 'mean']

# 스타일링 함수 정의
def highlight_minimum(s):
    is_min = s == s.min()  # 최소값 여부 확인
    return ['background-color: lightgreen' if v else '' for v in is_min]

In [52]:
print("\n전체 평균 모델 성능 통계(random41):")
display(model_stats_mean.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
))

print("\n전체 평균 모델 성능 통계 (배터리 용량 Not NaN):")
display(model_stats_notnan_mean)

print("\n전체 평균 모델 성능 통계 (배터리 용량 NaN):")
display(model_stats_isnan_mean)


전체 평균 모델 성능 통계(random41):



전체 평균 모델 성능 통계 (배터리 용량 Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.498175  0.360709  0.496694  0.385786  0.473342  0.337559     0.488464   
1   0.559489  0.337421  0.554199  0.360729  0.571474  0.329487     0.549174   
2   0.282137  0.190037  0.279309  0.263154  0.301563  0.215118     0.268868   
3   0.276527  0.175727  0.341458  0.168966  0.303734  0.205415     0.303690   
4   5.011027  3.362280  4.991391  2.940454  5.034413  3.270658     4.968227   
5   0.483736  0.336895  0.508733  0.345252  0.526358  0.359280     0.485261   
6   0.287201  0.218418  0.357845  0.314117  0.339229  0.241543     0.290930   
7   0.601604  0.355348  0.586931  0.399343  0.623663  0.389740     0.577917   
8   0.564804  0.401486  0.560666  0.401500  0.566552  0.342081     0.536229   
9   0.551618  0.368865  0.693955  0.514745  0.588118  0.421343     0.589385   
10  0.508821  0.316803  0.572359  0.406948  0.513366  0.323485     0.527315   
11  0.374004  0.326309  0.367564  0.287651  0.358269  0.266633     0.365578   
12  0.282545  0.202628  0.423810  0.417440  0.290695  0.224410     0.294945   
13  0.520399  0.307917  0.590760  0.398723  0.541076  0.324020     0.534199   
14  1.080248  1.535489  1.187547  1.342048  1.076032  1.433168     1.087700   
15  1.615691  1.608139  1.677419  1.402783  1.591065  1.486075     1.629445   
16  0.351393  0.321331  0.628083  0.460958  0.428118  0.370955     0.416612   
17  0.401864  0.295395  0.443469  0.311063  0.416077  0.311373     0.398680   
18  0.293655  0.183303  0.375152  0.286089  0.300831  0.195718     0.315941   
19  0.499181  0.319197  0.588155  0.403027  0.519923  0.340008     0.521919   
20  0.476776  0.323880  0.520706  0.333218  0.500466  0.323985     0.477368   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.360085     0.483908  0.344995     0.473922  0.351768         0.478642   
1   0.331681     0.561608  0.325101     0.553407  0.331620         0.552079   
2   0.216928     0.286436  0.196722     0.276062  0.227891         0.273967   
3   0.150225     0.284631  0.172523     0.310272  0.154156         0.295684   
4   2.998899     4.996892  3.210909     4.998958  2.998099         4.983261   
5   0.333958     0.499915  0.337905     0.504064  0.332420         0.492358   
6   0.258727     0.302394  0.219004     0.317723  0.250426         0.294976   
7   0.353675     0.604676  0.362386     0.590532  0.372452         0.586477   
8   0.386072     0.559327  0.344226     0.542673  0.333713         0.541552   
9   0.417923     0.558334  0.382448     0.601648  0.435419         0.570975   
10  0.332039     0.506300  0.315149     0.527339  0.335996         0.515547   
11  0.289120     0.362864  0.288615     0.356089  0.261405         0.360499   
12  0.266795     0.276581  0.203907     0.315741  0.276138         0.276542   
13  0.338712     0.524953  0.315044     0.544534  0.341026         0.528725   
14  1.403419     1.073248  1.469613     1.087227  1.371802         1.073253   
15  1.465084     1.601653  1.531706     1.613984  1.427132         1.609613   
16  0.324577     0.374220  0.319776     0.454883  0.330578         0.391317   
17  0.289358     0.401614  0.293716     0.404147  0.291157         0.390941   
18  0.204687     0.291621  0.186285     0.315353  0.206949         0.299612   
19  0.333366     0.506986  0.322360     0.530802  0.335928         0.516208   
20  0.315422     0.480340  0.326374     0.492921  0.316028         0.481634   

              
         std  
모델            
0   0.350081  
1   0.325222  
2   0.209755  
3   0.152034  
4   3.022827  
5   0.331373  
6   0.236489  
7   0.356696  
8   0.342942  
9   0.406665  
10  0.320731  



전체 평균 모델 성능 통계 (배터리 용량 NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.507434  0.333049  0.507271  0.326470  0.501005  0.320964     0.503879   
1   0.523373  0.329906  0.508055  0.321360  0.527371  0.336819     0.511224   
2   0.277983  0.185701  0.286968  0.227908  0.290559  0.189055     0.273730   
3   0.260049  0.186581  0.314420  0.142725  0.260665  0.165542     0.281880   
4   4.107058  2.737486  4.271139  2.757766  4.363476  2.680292     4.122648   
5   0.493329  0.318699  0.509200  0.395748  0.487401  0.319906     0.490415   
6   0.432145  0.328150  0.451174  0.271904  0.421581  0.323081     0.439359   
7   0.452347  0.272388  0.610498  0.395090  0.398934  0.311114     0.494016   
8   0.595151  0.315240  0.665082  0.435036  0.575139  0.372023     0.615095   
9   0.503429  0.352812  0.637681  0.445866  0.493537  0.328204     0.557687   
10  0.625600  0.325213  0.767035  0.432445  0.565137  0.285683     0.655447   
11  0.276706  0.184747  0.272807  0.222551  0.269535  0.204826     0.266554   
12  0.295102  0.209586  0.354889  0.293760  0.298805  0.224980     0.295280   
13  0.511352  0.346806  0.573025  0.328969  0.556916  0.325283     0.514232   
14  3.965882  1.789030  3.871643  0.830190  3.797937  1.351104     3.907411   
15  4.843834  2.052071  5.019425  0.827749  4.905430  1.598160     4.931630   
16  0.506593  0.443502  0.460363  0.353803  0.526323  0.406609     0.426509   
17  0.295871  0.173113  0.446222  0.302036  0.303576  0.202410     0.334515   
18  0.298831  0.261776  0.461091  0.311991  0.300565  0.223910     0.329047   
19  0.599333  0.457491  0.561509  0.323532  0.547593  0.367719     0.558089   
20  0.491495  0.310494  0.512724  0.307879  0.496991  0.320650     0.499800   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.322368     0.501264  0.325456     0.501119  0.318136         0.500946   
1   0.316808     0.521978  0.330366     0.510788  0.320392         0.512921   
2   0.203666     0.282634  0.184311     0.277587  0.202592         0.275839   
3   0.152706     0.259288  0.170043     0.283456  0.145199         0.273725   
4   2.721723     4.217392  2.632944     4.283531  2.666088         4.198970   
5   0.341785     0.489334  0.317263     0.485934  0.343956         0.486171   
6   0.289258     0.421415  0.320920     0.428873  0.290051         0.427709   
7   0.331289     0.413751  0.284176     0.465629  0.334072         0.441656   
8   0.355763     0.583909  0.334428     0.607649  0.372773         0.598441   
9   0.376222     0.492796  0.317452     0.536016  0.351093         0.522164   
10  0.370983     0.591447  0.301159     0.616838  0.356371         0.611183   
11  0.198243     0.270301  0.189129     0.264618  0.209379         0.265715   
12  0.244963     0.292209  0.212033     0.296642  0.250575         0.288319   
13  0.320350     0.523291  0.318037     0.538967  0.315438         0.514978   
14  1.180956     3.880858  1.485492     3.832078  0.990170         3.870919   
15  1.335886     4.874632  1.745558     4.962428  1.139162         4.922896   
16  0.307770     0.491178  0.385080     0.456289  0.301417         0.448508   
17  0.199496     0.299224  0.172512     0.343011  0.212931         0.317367   
18  0.288697     0.290803  0.230555     0.337377  0.283035         0.298510   
19  0.362417     0.540990  0.357725     0.545718  0.327981         0.537065   
20  0.294903     0.490086  0.308813     0.497849  0.299039         0.494146   

              
         std  
모델            
0   0.320510  
1   0.320294  
2   0.195067  
3   0.152669  
4   2.648590  
5   0.329651  
6   0.298374  
7   0.318603  
8   0.347338  
9   0.340087  
10  0.335221  


In [34]:
print("\n전체 평균 모델 성능 통계(random43):")
display(model_stats_mean.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
))

print("\n전체 평균 모델 성능 통계 (배터리 용량 Not NaN):")
display(model_stats_notnan_mean)

print("\n전체 평균 모델 성능 통계 (배터리 용량 NaN):")
display(model_stats_isnan_mean)


전체 평균 모델 성능 통계(random43):



전체 평균 모델 성능 통계 (배터리 용량 Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.536032  0.329778  0.554890  0.369468  0.516530  0.327380     0.528564   
1   0.544915  0.317095  0.569195  0.355495  0.571009  0.344720     0.544507   
2   0.285106  0.196269  0.328041  0.282592  0.295793  0.229275     0.280517   
3   0.298535  0.216631  0.341740  0.181146  0.316653  0.197104     0.311875   
4   5.308314  3.027982  5.107952  2.816313  5.313438  3.231496     5.177506   
5   0.487224  0.361091  0.535405  0.401028  0.500743  0.320885     0.497964   
6   0.279322  0.227846  0.373183  0.310876  0.326043  0.236482     0.298839   
7   0.602901  0.405435  0.577005  0.404751  0.613870  0.413323     0.571570   
8   0.536463  0.322410  0.542188  0.391763  0.556366  0.372517     0.525578   
9   0.539393  0.362349  0.713018  0.529713  0.563380  0.381343     0.595179   
10  0.515233  0.310284  0.561382  0.366367  0.520869  0.331664     0.520527   
11  0.349485  0.262619  0.396935  0.299350  0.347780  0.250751     0.350469   
12  0.319159  0.227354  0.426336  0.403968  0.299217  0.246555     0.312173   
13  0.537103  0.313026  0.655074  0.428923  0.535124  0.321919     0.563028   
14  1.081186  1.575489  1.289743  1.371905  1.083082  1.494593     1.128284   
15  1.457787  1.309761  1.690915  1.322117  1.483610  1.279646     1.557229   
16  0.346271  0.333942  0.724334  0.654190  0.423537  0.390076     0.460893   
17  0.406514  0.318171  0.464670  0.323778  0.427709  0.326520     0.415224   
18  0.288515  0.184010  0.401801  0.312436  0.289053  0.192561     0.317600   
19  0.517793  0.325214  0.658246  0.454322  0.519365  0.325099     0.549295   
20  0.511549  0.282335  0.548058  0.313724  0.533054  0.308522     0.515943   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.340409     0.523315  0.317687     0.520485  0.339434         0.522342   
1   0.322799     0.555363  0.323794     0.556207  0.331076         0.548857   
2   0.233840     0.284988  0.205791     0.285029  0.242829         0.275731   
3   0.171321     0.297015  0.182403     0.310458  0.165482         0.302550   
4   2.747870     5.304230  2.984593     5.191997  2.932270         5.219654   
5   0.354723     0.489152  0.329997     0.498676  0.342017         0.488776   
6   0.259260     0.294443  0.218785     0.310107  0.253219         0.294488   
7   0.377665     0.605205  0.397665     0.581355  0.381139         0.583626   
8   0.335893     0.535894  0.332291     0.529575  0.336132         0.524258   
9   0.419635     0.543227  0.359357     0.605005  0.421011         0.572278   
10  0.321116     0.513095  0.315381     0.518450  0.325764         0.512554   
11  0.273601     0.345778  0.249971     0.348693  0.262390         0.340823   
12  0.279875     0.294421  0.228729     0.309706  0.286904         0.286385   
13  0.351565     0.533183  0.309547     0.561784  0.353584         0.543812   
14  1.433261     1.072845  1.526744     1.129104  1.410254         1.098136   
15  1.280240     1.464313  1.278509     1.566382  1.283194         1.524732   
16  0.401862     0.360256  0.320062     0.498338  0.418332         0.415381   
17  0.297143     0.408782  0.311250     0.422861  0.292839         0.408836   
18  0.222434     0.285842  0.182904     0.316228  0.226426         0.297929   
19  0.362493     0.513783  0.319617     0.549043  0.359519         0.526802   
20  0.268880     0.519180  0.282638     0.524760  0.271780         0.515277   

              
         std  
모델            
0   0.326306  
1   0.321075  
2   0.225161  
3   0.162298  
4   2.840312  
5   0.338289  
6   0.234860  
7   0.375991  
8   0.326134  
9   0.389707  
10  0.314606  



전체 평균 모델 성능 통계 (배터리 용량 NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.516361  0.333627  0.514032  0.330439  0.513118  0.332991     0.508926   
1   0.517218  0.326549  0.509040  0.308349  0.511997  0.337408     0.505727   
2   0.291643  0.198444  0.306541  0.265356  0.297369  0.188989     0.283403   
3   0.272658  0.179671  0.336702  0.153023  0.256467  0.180642     0.296667   
4   4.872503  3.516460  4.997511  2.972636  4.668908  2.899051     4.871142   
5   0.497488  0.317355  0.499878  0.344632  0.495221  0.323035     0.491376   
6   0.449872  0.325990  0.411226  0.302030  0.447658  0.307464     0.426210   
7   0.426957  0.305697  0.621150  0.394944  0.405127  0.252495     0.506607   
8   0.637064  0.376337  0.753994  0.534527  0.607348  0.380999     0.675678   
9   0.493956  0.332537  0.591710  0.455411  0.512490  0.348644     0.524138   
10  0.587100  0.329234  0.718849  0.436737  0.613646  0.314849     0.603115   
11  0.269610  0.190641  0.316039  0.247293  0.264931  0.196016     0.258351   
12  0.300310  0.218234  0.409326  0.339706  0.289945  0.212630     0.323608   
13  0.561842  0.367319  0.599931  0.417610  0.549761  0.361132     0.554111   
14  4.023804  1.547136  3.874122  0.833492  3.870935  1.391538     3.944676   
15  4.790905  1.905023  4.943032  0.714221  4.775573  1.468780     4.866969   
16  0.478027  0.434426  0.422521  0.330914  0.545904  0.447918     0.398787   
17  0.257818  0.172394  0.488927  0.409862  0.310555  0.192050     0.334864   
18  0.494814  0.538021  0.382628  0.291886  0.789274  1.065917     0.408922   
19  0.568118  0.425264  0.552253  0.312471  0.524554  0.355504     0.527828   
20  0.492388  0.298543  0.505951  0.294123  0.499205  0.304290     0.497200   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.324724     0.511356  0.331179     0.508114  0.327805         0.507975   
1   0.312621     0.509875  0.330727     0.503345  0.311324         0.503977   
2   0.226948     0.291887  0.186209     0.286651  0.214856         0.285281   
3   0.148886     0.261761  0.176162     0.288184  0.151298         0.280191   
4   3.204739     4.765699  3.122764     4.774351  2.937246         4.803731   
5   0.315774     0.494787  0.318621     0.489864  0.318414         0.488823   
6   0.304254     0.447154  0.306466     0.423477  0.296337         0.430420   
7   0.295780     0.400962  0.263752     0.466026  0.288720         0.443145   
8   0.426060     0.621822  0.365416     0.662114  0.424644         0.649077   
9   0.381053     0.493065  0.329152     0.533475  0.359019         0.509858   
10  0.387357     0.594154  0.322842     0.621759  0.367388         0.594478   
11  0.222166     0.262693  0.190973     0.259008  0.226444         0.252440   
12  0.260676     0.289906  0.209118     0.320161  0.262907         0.303573   
13  0.348066     0.548087  0.350373     0.546260  0.359378         0.540507   
14  1.074070     3.943082  1.396044     3.872529  1.007272         3.920096   
15  1.196464     4.783239  1.614180     4.859303  1.003361         4.836504   
16  0.331977     0.486570  0.405331     0.424158  0.330485         0.418000   
17  0.238237     0.278545  0.174634     0.362521  0.279774         0.319305   
18  0.381419     0.635651  0.793232     0.554781  0.638224         0.522926   
19  0.343181     0.517163  0.376230     0.524411  0.314400         0.518608   
20  0.286350     0.491619  0.294500     0.498081  0.286259         0.493802   

              
         std  
모델            
0   0.326147  
1   0.316624  
2   0.203888  
3   0.154610  
4   3.051435  
5   0.316232  
6   0.299836  
7   0.277014  
8   0.396847  
9   0.350642  
10  0.358137  


In [28]:
print("\n전체 평균 모델 성능 통계:")
display(model_stats_mean.style.apply(
    highlight_minimum,
    subset=mean_columns,
    axis=1  # 행별로 적용
))

print("\n전체 평균 모델 성능 통계 (배터리 용량 Not NaN):")
display(model_stats_notnan_mean)

print("\n전체 평균 모델 성능 통계 (배터리 용량 NaN):")
display(model_stats_isnan_mean)


전체 평균 모델 성능 통계:



전체 평균 모델 성능 통계 (배터리 용량 Not NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.531871  0.314803  0.545022  0.388487  0.520511  0.322140     0.529738   
1   0.531073  0.333774  0.558853  0.357985  0.567846  0.355799     0.535127   
2   0.271558  0.196044  0.289633  0.265569  0.302373  0.223307     0.262884   
3   0.276279  0.181206  0.354462  0.187455  0.293039  0.187759     0.308432   
4   5.256622  3.318481  5.102136  2.832181  5.115035  3.145113     5.134030   
5   0.476720  0.323562  0.506090  0.346257  0.496503  0.343833     0.478947   
6   0.328783  0.261655  0.359100  0.325791  0.340258  0.261251     0.320368   
7   0.600402  0.414156  0.592705  0.428626  0.597607  0.369064     0.578975   
8   0.555934  0.357559  0.580533  0.407379  0.611435  0.455436     0.545520   
9   0.546770  0.372394  0.713389  0.495421  0.587647  0.414678     0.601936   
10  0.504651  0.312241  0.562144  0.355841  0.514577  0.331162     0.524029   
11  0.352769  0.270111  0.416214  0.309656  0.349369  0.253819     0.361104   
12  0.304179  0.214027  0.470718  0.411959  0.319833  0.247775     0.339296   
13  0.535830  0.301065  0.629143  0.430368  0.529549  0.317878     0.553783   
14  1.132942  1.662283  1.246612  1.393538  1.058791  1.469631     1.146774   
15  1.497092  1.508238  1.662240  1.350474  1.495409  1.459651     1.561613   
16  0.352209  0.342818  0.689259  0.532851  0.414739  0.346925     0.437781   
17  0.395036  0.294670  0.459003  0.323665  0.435613  0.420286     0.402175   
18  0.285191  0.178928  0.403547  0.326076  0.286848  0.187005     0.319047   
19  0.500156  0.307769  0.603746  0.411017  0.523738  0.319754     0.525162   
20  0.515335  0.341267  0.593502  0.347845  0.549324  0.310540     0.542604   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.329869     0.519108  0.321395     0.526747  0.334515         0.521974   
1   0.328649     0.544234  0.336268     0.546950  0.343201         0.538770   
2   0.226907     0.278446  0.205990     0.277230  0.234801         0.268168   
3   0.156102     0.278307  0.171702     0.313095  0.160036         0.298612   
4   2.946926     5.143341  3.158102     5.106913  2.882205         5.125229   
5   0.327531     0.481691  0.327191     0.485158  0.326820         0.479643   
6   0.271314     0.318288  0.253363     0.316968  0.255514         0.309962   
7   0.390183     0.595403  0.373907     0.581873  0.370673         0.582100   
8   0.368848     0.576728  0.377980     0.573526  0.389773         0.559835   
9   0.411176     0.558097  0.382209     0.615623  0.428393         0.584167   
10  0.310132     0.506598  0.315769     0.524523  0.318973         0.514975   
11  0.276612     0.348675  0.254632     0.359250  0.269224         0.349589   
12  0.264639     0.304393  0.226019     0.352999  0.282935         0.317379   
13  0.343623     0.529963  0.302007     0.553584  0.345908         0.536733   
14  1.487507     1.090682  1.559371     1.108430  1.397127         1.104144   
15  1.387647     1.487873  1.470574     1.559409  1.383462         1.532014   
16  0.338162     0.366585  0.311003     0.463746  0.350938         0.397586   
17  0.286325     0.406061  0.333551     0.422176  0.326319         0.399859   
18  0.227872     0.282503  0.176904     0.317031  0.224349         0.297534   
19  0.327929     0.508169  0.311258     0.535122  0.336190         0.515521   
20  0.319726     0.527630  0.322138     0.556390  0.293770         0.538340   

              
         std  
모델            
0   0.325312  
1   0.331091  
2   0.219602  
3   0.151733  
4   2.948030  
5   0.321292  
6   0.251680  
7   0.368621  
8   0.365508  
9   0.397689  
10  0.309027  



전체 평균 모델 성능 통계 (배터리 용량 NaN):


xgb-real            cat-real            lgb-real           xgb_cat-real  \
        mean       std      mean       std      mean       std         mean   
모델                                                                            
0   0.523006  0.327564  0.512454  0.327228  0.505161  0.324060     0.509986   
1   0.519171  0.320533  0.510258  0.312000  0.528876  0.321605     0.507098   
2   0.282667  0.199869  0.300771  0.236900  0.289840  0.194909     0.274702   
3   0.251133  0.179543  0.330824  0.167439  0.261719  0.176951     0.282798   
4   4.722204  2.950593  4.560706  2.588946  4.849144  2.780401     4.606291   
5   0.501656  0.337325  0.523703  0.363556  0.501809  0.333632     0.506104   
6   0.458124  0.346513  0.462581  0.323223  0.419427  0.305901     0.448196   
7   0.475988  0.404226  0.622195  0.324242  0.438676  0.313767     0.520848   
8   0.616585  0.384714  0.701146  0.443253  0.565756  0.333210     0.649789   
9   0.525654  0.349664  0.671784  0.530035  0.519341  0.324521     0.575390   
10  0.632173  0.369265  0.816522  0.515493  0.603828  0.300920     0.658096   
11  0.275583  0.186915  0.326463  0.258419  0.270567  0.199336     0.274400   
12  0.294864  0.210239  0.367799  0.285654  0.308557  0.217008     0.302815   
13  0.611726  0.380728  0.608539  0.388816  0.567010  0.351119     0.573509   
14  3.883205  1.629403  3.868572  0.829114  3.863714  1.458236     3.870374   
15  5.020995  2.183068  5.029021  0.807260  4.984979  1.783491     5.025008   
16  0.480185  0.397326  0.389775  0.306318  0.629733  0.494701     0.387580   
17  0.301057  0.186888  0.492849  0.297687  0.288034  0.177600     0.387093   
18  0.313524  0.258349  0.522098  0.387919  0.356866  0.235739     0.399477   
19  0.548303  0.414469  0.573057  0.309260  0.572739  0.328550     0.534327   
20  0.503800  0.323253  0.518126  0.332857  0.522060  0.311516     0.503790   

             xgb_lgb-real           cat_lgb-real           xgb_cat_lgb-real  \
         std         mean       std         mean       std             mean   
모델                                                                            
0   0.320312     0.510534  0.324772     0.504028  0.321370         0.506786   
1   0.311260     0.521629  0.317284     0.512384  0.312145         0.511878   
2   0.212716     0.284103  0.190841     0.274527  0.211753         0.275108   
3   0.155116     0.253646  0.173431     0.287576  0.155237         0.273186   
4   2.722409     4.753211  2.844299     4.676038  2.641004         4.663174   
5   0.335423     0.498143  0.336716     0.500838  0.339696         0.499828   
6   0.320830     0.431847  0.311736     0.422399  0.303787         0.429572   
7   0.313895     0.432484  0.349420     0.465001  0.290342         0.461534   
8   0.375040     0.588767  0.345732     0.631287  0.351097         0.619924   
9   0.426080     0.516837  0.323085     0.558631  0.397444         0.544338   
10  0.400841     0.615851  0.322978     0.658236  0.367519         0.622511   
11  0.225413     0.267482  0.190429     0.272232  0.228195         0.266064   
12  0.238268     0.297968  0.204607     0.306788  0.240924         0.295309   
13  0.344956     0.583464  0.343817     0.562597  0.321122         0.563918   
14  1.112888     3.869291  1.495448     3.866143  1.030501         3.868154   
15  1.388888     5.002987  1.914500     5.007000  1.211012         5.011665   
16  0.285235     0.518874  0.415580     0.449675  0.331262         0.434725   
17  0.197605     0.291490  0.174533     0.382966  0.199665         0.350958   
18  0.299682     0.323250  0.227466     0.427779  0.289634         0.373823   
19  0.330910     0.534494  0.350405     0.553740  0.312347         0.528781   
20  0.315324     0.505438  0.313964     0.509489  0.311520         0.503335   

              
         std  
모델            
0   0.320109  
1   0.312019  
2   0.200729  
3   0.155094  
4   2.729500  
5   0.333752  
6   0.308192  
7   0.301139  
8   0.347790  
9   0.371964  
10  0.356519  
